In [17]:
import ee
import ee.data
import json
from gee_toolbox.gee import assets
import pprint

In [2]:
config = {
    "service_credentials_file": "../secrets/ee-observatorionieves-288939dbc1cf.json"
}

with open(config['service_credentials_file'], "r") as f:
        service_account_data = json.load(f)
service_user = service_account_data["client_email"]
print(service_user)

credentials = ee._helpers.ServiceAccountCredentials(
    email=service_user,
    key_data=json.dumps(service_account_data),
)
ee.Initialize(credentials, project='ee-observatorionieves')

osn-imageautomation-dev@ee-observatorionieves.iam.gserviceaccount.com


In [22]:
def copy_img_collection(source_ic, dest_ic):
    # check if dest_ic already exists. End if it does, create if it doesn't
    try:
        ee.data.getAsset(dest_ic)
        print(f"{dest_ic} already exists. Exiting.")
        return
    except Exception as e:
        print(f"Creating {dest_ic}")
        ee.data.createAsset(value={'type': 'IMAGE_COLLECTION'}, path=dest_ic)

    ic_images = assets.list_assets(
        parent=source_ic,
        asset_types='Image',
        expand_image_collections=True
        )

    ic_images = assets.get_asset_names(ic_images)

    for _image in ic_images:
        # Get image name
        image_name = _image.split('/')[-1]
        # Create new path+name
        new_path = f'{dest_ic}/{image_name}'
        ee.data.copyAsset(_image, new_path)

    return

In [23]:
# Copy Monthly Image Collection
source_ic = 'users/observatorionieves/MODIS/Andes_MCDS4S5_Yearly_Monthly'
dest_ic = 'projects/ee-observatorionieves/assets/MODIS/Andes_MCDS4S5_Yearly_Monthly'
copy_img_collection(source_ic, dest_ic)

Creating projects/ee-observatorionieves/assets/MODIS/Andes_MCDS4S5_Yearly_Monthly


In [ ]:
# Copy Yearly Image Collection
source_ic = "users/observatorionieves/MODIS/Andes_MCDS4S5_Yearly"
dest_ic = "projects/ee-observatorionieves/assets/MODIS/Andes_MCDS4S5_Yearly"
copy_img_collection(source_ic, dest_ic)